## 0. System Information Beforehand

In [1]:
uname -r
lsb_release -a

6.1.28
LSB Version:	core-11.1.0ubuntu4-noarch:security-11.1.0ubuntu4-noarch
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.2 LTS
Release:	22.04
Codename:	jammy


Install additional packages before building a kernel:

In [ ]:
sudo apt-get install -y --no-install-recommends fakeroot \
                                                build-essential \
                                                ncurses-dev \
                                                xz-utils \
                                                libssl-dev \
                                                bc \
                                                flex \
                                                libelf-dev \
                                                bison

## 1. Get Linux Kernel Source Tree

Download package from the linux kernel archives website:

In [2]:
curl -o /tmp/linux-6.1.29.tar.xz https://cdn.kernel.org/pub/linux/kernel/v6.x/linux-6.1.29.tar.xz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  128M  100  128M    0     0  4955k      0  0:00:26  0:00:26 --:--:-- 5656k


Extract source files:

In [3]:
tar -xf /tmp/linux-6.1.29.tar.xz -C $HOME/Studio/

Backup source tree(optional, btrfs only):

In [4]:
btrfs subvolume snapshot $HOME/Studio/linux-6.1.29 $HOME/GitHub/@linux-6.1.29

Create a snapshot of '/home/clymber/Studio/linux-6.1.29' in '/home/clymber/GitHub/@linux-6.1.29'


After making changes to your configuration file, or when using an existing configuration file on a new kernel tree, you can validate and update the configuration: `$ make oldconfig`. You should always run this before building a kernel.

In [ ]:
# execute it in the system terminal
make oldconfig

## 2. Configure Kernel

In [9]:
cd /home/clymber/Studio/linux-6.1.29

/home/clymber/Studio/linux-6.1.29


For this time, we're going to copy the existed configuration file from current system to simplify the work. The current system kernel configuration  usually locates at `/boot//boot/config-<kernel-release>`.

In [11]:
cp -v /boot/config-$(uname -r) .config

'/boot/config-6.1.28' -> '.config'


We can use the `scripts/config` within the source tree to check/change options for compiling the kernel.

In [13]:
opts="BPF HAVE_EBPF_JIT BPF_SYSCALL BPF_JIT IKHEADERS NET_CLS_BPF NET_ACT_BPF \
      BPF_EVENTS TRUSTED_KEYS REVOCATION_KEYS"
for opt in $opts; do
    echo "CONFIG_${opt}="$(scripts/config --state "CONFIG_${opt}")
done

CONFIG_BPF=y
CONFIG_HAVE_EBPF_JIT=y
CONFIG_BPF_SYSCALL=y
CONFIG_BPF_JIT=y
CONFIG_IKHEADERS=m
CONFIG_NET_CLS_BPF=m
CONFIG_NET_ACT_BPF=m
CONFIG_BPF_EVENTS=y
CONFIG_TRUSTED_KEYS=y
CONFIG_REVOCATION_KEYS=undef


In [18]:
scripts/config --enable "CONFIG_IKHEADERS"

If you are compiling the kernel on Ubuntu, you may receive the following error that interrupts the building process: `No rule to make target 'debian/canonical-certs.pem`, disable the conflicting security certificates by executing the two commands below:

In [15]:
scripts/config --disable "CONFIG_TRUSTED_KEYS"
scripts/config --disable "CONFIG_REVOCATION_KEYS"

## 3. Compling

please DO NOT execute the following command in jupyter notebook if not necessary, because it will consume large amount of memory and slow down the system. Do it with the system shell terminal to have better performance.

In [ ]:
make -j8

## 4. Install the required modules

In [4]:
sudo make -j8 modules_install

  INSTALL /lib/modules/6.1.29/kernel/sound/soc/intel/boards/snd-soc-sst-byt-cht-es8316.ko
  SIGN    /lib/modules/6.1.29/kernel/sound/soc/intel/boards/snd-soc-sst-byt-cht-da7213.ko
  SIGN    /lib/modules/6.1.29/kernel/sound/soc/intel/boards/snd-soc-sst-bxt-rt298.ko
  SIGN    /lib/modules/6.1.29/kernel/sound/soc/intel/boards/snd-soc-sst-byt-cht-cx2072x.ko
  INSTALL /lib/modules/6.1.29/kernel/sound/soc/intel/boards/snd-soc-sst-bytcr-rt5640.ko
  INSTALL /lib/modules/6.1.29/kernel/sound/soc/intel/boards/snd-soc-sst-bytcr-rt5651.ko
  SIGN    /lib/modules/6.1.29/kernel/sound/soc/intel/boards/snd-soc-sst-byt-cht-es8316.ko
  INSTALL /lib/modules/6.1.29/kernel/sound/soc/intel/boards/snd-soc-sst-bytcr-wm5102.ko
  INSTALL /lib/modules/6.1.29/kernel/sound/soc/intel/boards/snd-soc-sst-cht-bsw-max98090_ti.ko
  INSTALL /lib/modules/6.1.29/kernel/sound/soc/intel/boards/snd-soc-sst-cht-bsw-nau8824.ko
  INSTALL /lib/modules/6.1.29/kernel/sound/soc/intel/boards/snd-soc-sst-cht-bsw-rt5645.ko
  SIGN    /lib

In [5]:
du -sh /lib/modules/*

604M	/lib/modules/5.19.0-41-generic
397M	/lib/modules/6.1.28
5.8G	/lib/modules/6.1.29


## 5. Install the kernel

In [6]:
sudo make install

  INSTALL /boot
run-parts: executing /etc/kernel/postinst.d/dkms 6.1.29 /boot/vmlinuz-6.1.29
 * dkms: running auto installation service for kernel 6.1.29        
Kernel preparation unnecessary for this kernel. Skipping...
applying patch 0002-Makefile.patch...patching file Makefile
Hunk #1 succeeded at 113 with fuzz 1.
Hunk #2 succeeded at 132 with fuzz 2 (offset 1 line).

applying patch 0003-Make-up-for-missing-init_MUTEX.patch...patching file src/wl/sys/wl_linux.c
Hunk #1 succeeded at 111 with fuzz 2 (offset 12 lines).

applying patch 0010-change-the-network-interface-name-from-eth-to-wlan.patch...patching file src/wl/sys/wl_linux.c
Hunk #1 succeeded at 221 (offset -14 lines).

applying patch 0013-gcc.patch...patching file Makefile

applying patch 0019-broadcom-sta-6.30.223.248-3.18-null-pointer-fix.patch...patching file src/wl/sys/wl_linux.c
Hunk #1 succeeded at 2169 (offset 12 lines).

applying patch 0020-add-support-for-linux-4.3.patch...patching file src/shared/linux_osl.c

applyi

## 6. Delete debug infomation for kernel modules(Optional)
For this version, 6.1.19, we are not going to strip the debug information for the modules now. The following is the history records of installing 6.2.0 kernel.

The 6.2 version was tagged with EOL, short for End of Life, in the kernel archives website. But 6.1.19 is a long term support version, so I would like to keep it as a backup kernel.
After the kernel modules installed, it turned out that the new modules occupied too much disk drive space.

In [1]:
du -sh /lib/modules/*

603M	/lib/modules/5.19.0-32-generic
604M	/lib/modules/5.19.0-41-generic
6.0G	/lib/modules/6.2.0


In [3]:
file /lib/modules/{5.19.0-41-generic,6.2.0}/kernel/fs/btrfs/btrfs.ko
ls -lh /lib/modules/{5.19.0-41-generic,6.2.0}/kernel/fs/btrfs/btrfs.ko

/lib/modules/5.19.0-41-generic/kernel/fs/btrfs/btrfs.ko: ELF 64-bit LSB relocatable, x86-64, version 1 (SYSV), BuildID[sha1]=0e2afdf8bcd0fe6a6e39ae89795f923156aa5565, not stripped
/lib/modules/6.2.0/kernel/fs/btrfs/btrfs.ko:             ELF 64-bit LSB relocatable, x86-64, version 1 (SYSV), BuildID[sha1]=d8002917f3a35a364ae485d767c07d99652271b8, with debug_info, not stripped
-rw-r--r-- 1 root root 3.1M Apr 18 23:38 /lib/modules/5.19.0-41-generic/kernel/fs/btrfs/btrfs.ko
-rw-r--r-- 1 root root  35M May 11 00:18 /lib/modules/6.2.0/kernel/fs/btrfs/btrfs.ko


From first command output above, we can see that the newly installed kernel '6.2.0' is more than 10 times larger the the preinstalled version. The next command outputs showed that, the module files in the new version is 'with debug_info', and that's the reason.

Keep in mind that beyond the use of disk space, this is not as significant as it may appear. Debugging symbols are not loaded during normal runtime, so the actual size of each module in memory is probably identical regardless of the size of the .ko file.

Anyway, if we want to minimize the use of disk space, the `strip` command:

In [4]:
strip --strip-all "./fs/btrfs/btrfs.ko" -o "./fs/btrfs/btrfs.ko.stripped"

In [5]:
ls -lh ./fs/btrfs/btrfs.ko{,.stripped}
file ./fs/btrfs/btrfs.ko{,.stripped}

-rw-rw-r-- 1 clymber clymber  35M May 10 23:52 ./fs/btrfs/btrfs.ko
-rw-rw-r-- 1 clymber clymber 1.5M May 11 14:50 ./fs/btrfs/btrfs.ko.stripped
./fs/btrfs/btrfs.ko:          ELF 64-bit LSB relocatable, x86-64, version 1 (SYSV), BuildID[sha1]=d8002917f3a35a364ae485d767c07d99652271b8, with debug_info, not stripped
./fs/btrfs/btrfs.ko.stripped: ELF 64-bit LSB relocatable, x86-64, version 1 (SYSV), BuildID[sha1]=d8002917f3a35a364ae485d767c07d99652271b8, stripped


As showed above, the file size with its debug information stripped is much smaller the the original one.

Alternatively, we can avoid the toiling stripping work during kernel modules installation, like: 

In [ ]:
# DO NOT execute the follow command in jupyter notebook if not necessary,
# it will consume large amount of memory and slow down the system.
# Do it with the system shell terminal to have better performance.
sudo make INSTALL_MOD_STRIP=1 modules_install

In [8]:
du -sh /lib/modules/*

603M	/lib/modules/5.19.0-32-generic
604M	/lib/modules/5.19.0-41-generic
467M	/lib/modules/6.2.0


After stripped, the disk space consumed by the new kernel is significantly smaller, from 6 GB to 467 M!

## 7. Update the Bootloader (Optional)

The GRUB bootloader is the first program that runs when the system powers on.  
The make install command performs this process automatically, but you can also do it manually.

### 1. Update the initramfs to the installed kernel version:

In [6]:
sudo update-initramfs -c -k 6.2.0

update-initramfs: Generating /boot/initrd.img-6.2.0
I: The initramfs will attempt to resume from /dev/sda2
I: (UUID=3bef089a-1f68-482e-9ba3-9c51d70e3d07)
I: Set the RESUME variable to override this.


### 2. Update the GRUB bootloader with this command:

In [7]:
sudo update-grub

Sourcing file `/etc/default/grub'
Sourcing file `/etc/default/grub.d/init-select.cfg'
Generating grub configuration file ...
Found linux image: /boot/vmlinuz-6.2.0
Found initrd image: /boot/initrd.img-6.2.0
Found linux image: /boot/vmlinuz-5.19.0-41-generic
Found initrd image: /boot/initrd.img-5.19.0-41-generic
Found linux image: /boot/vmlinuz-5.19.0-32-generic
Found initrd image: /boot/initrd.img-5.19.0-32-generic
Memtest86+ needs a 16-bit boot, that is not available on EFI, exiting
Systems on them will not be added to the GRUB boot configuration.
Check GRUB_DISABLE_OS_PROBER documentation entry.
Adding boot menu entry for UEFI Firmware Settings ...
done


## Reboot and Verify Kernel Version

When you complete the steps above, reboot the machine.  
When the system boots up, verify the kernel version using the uname command:

In [1]:
uname -mrs

Linux 6.2.0 x86_64
